In [1]:
from distributed import Client as DaskClient
import sqlalchemy as sa

from classify.features import *

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s - %(name)s.py]: %(message)s'
)

In [3]:
DaskClient()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 117.93 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43963,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 117.93 GiB
Comm: tcp://127.0.0.1:46855,Total threads: 4
Dashboard: http://127.0.0.1:39571/status,Memory: 29.48 GiB
Nanny: tcp://127.0.0.1:38261,


In [4]:
pd.set_option('max_columns', None)

In [5]:
config = ExpungeConfig.from_yaml('classify/expunge_config.yaml')

config.years_until_next_conviction_short

[INFO - classify.config.py]: Loading config from file: classify/expunge_config.yaml


numpy.timedelta64(7,'Y')

In [7]:
query = (
    sa.select(expunge)
    .where(
        sa.and_(
            expunge.c.HearingDate < '2262-04-11',
#             sa.or_(
#                 expunge.c.person_id == 1021000000606,
#                 expunge.c.person_id == 1070000000994,
#                 expunge.c.person_id == 163010000000348
#             )
        )
    )
    .order_by(
        expunge.c.person_id, 
        expunge.c.HearingDate
    )
)

In [8]:
%%time
ddf = fetch_expunge_data(
    custom_query=query,
#     n_partitions=32
)

ddf.head()

[INFO - classify.features.py]: Reading from table: expunge_clean


CPU times: user 1.32 s, sys: 296 ms, total: 1.61 s
Wall time: 30.7 s


,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
person_id,,,,,,,,,
1000000000002,2013-04-04,C.46.2-862,Misdemeanor,1,Guilty,Guilty,Unknown,Male,107
1000000000003,2015-10-21,A.46.2-853,Misdemeanor,NA,Nolle Prosequi,NA,White,Male,153
1000000000004,2009-10-07,A.46.2-862,Misdemeanor,1,Guilty,Guilty,Unknown,Male,87
1000000000006,2003-04-14,MISSING,Misdemeanor,NA,Dismissed,NA,Black,Male,700
1000000000008,2014-02-05,23-26,Misdemeanor,1,Nolle Prosequi,NA,Unknown,Male,740


In [ ]:
ddf = clean_data(ddf)

ddf.head()

In [ ]:
%%time
ddf = (
    ddf.pipe(build_disposition)
       .pipe(build_chargetype)
       .pipe(build_codesection, config)
       .pipe(build_convictions)
       .pipe(build_date_features)
       .pipe(build_timedelta_features, config)
       .pipe(build_class_features)
)

ddf.head()

In [10]:
ddf = remove_unneeded_columns(ddf)
ddf, run_id = append_run_id(ddf)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips,disposition,chargetype,codesection,convictions,last_hearing_date,last_felony_conviction_date,next_conviction_date,last_hearing_delta,last_felony_conviction_delta,next_conviction_delta,from_present_delta,arrest_disqualifier,felony_conviction_disqualifier,next_conviction_disqualifier_short,next_conviction_disqualifier_long,from_present_disqualifier_short,from_present_disqualifier_long,class1_2,class3_4,run_id
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1021000000606,2009-05-26,46.2-300,Misdemeanor,2,Guilty,NA,White,Female,169,Conviction,Misdemeanor,covered elsewhere,True,NaT,NaT,2011-09-08,NaN,NaN,835.0,4237,False,False,True,True,False,False,False,False,ed566fde-2e68-4c6e-a71c-a219f82808a0
1021000000606,2009-07-23,18.2-57,Misdemeanor,1,Dismissed,NA,White,Female,520,Dismissed,Misdemeanor,covered elsewhere,True,2009-05-26,NaT,2011-09-08,58.0,NaN,777.0,4179,True,False,True,True,False,False,False,False,ed566fde-2e68-4c6e-a71c-a219f82808a0
1021000000606,2011-09-08,18.2-250.1,Misdemeanor,U,Guilty,Guilty,White,Female,520,Conviction,Misdemeanor,covered in 19.2-392.6 - B,True,2009-07-23,NaT,2014-08-28,777.0,NaN,1085.0,3402,True,False,True,True,False,True,False,False,ed566fde-2e68-4c6e-a71c-a219f82808a0
1021000000606,2014-08-28,18.2-56.1,Misdemeanor,1,Guilty,Guilty,White,Female,520,Conviction,Misdemeanor,covered elsewhere,True,2011-09-08,NaT,2018-09-18,1085.0,NaN,1482.0,2317,True,False,True,True,True,True,False,False,ed566fde-2e68-4c6e-a71c-a219f82808a0
1021000000606,2018-09-18,18.2-248,Felony,U,Guilty,NA,White,Female,520,Conviction,Felony,covered elsewhere,True,2014-08-28,NaT,NaT,1482.0,NaN,NaN,835,False,False,False,False,True,True,False,False,ed566fde-2e68-4c6e-a71c-a219f82808a0


In [12]:
%%time
file_paths = write_to_csv(ddf)

[INFO - classify.features.py]: Expungement feature data will be written to: /tmp/expunge_data
[INFO - classify.features.py]: Clearing any data from previous runs
[INFO - classify.features.py]: Command 'rm -rf /tmp/expunge_data/expunge_features-*.csv' returned with exit value: 0
[INFO - classify.features.py]: Executing Dask task graph and writing results to CSV...
[INFO - classify.features.py]: File(s) written successfully


CPU times: user 291 ms, sys: 39.6 ms, total: 330 ms
Wall time: 478 ms


In [13]:
file_paths[:5]

['/tmp/expunge_data/expunge_features-0.csv']

In [14]:
%%time
load_to_db(file_paths)

[INFO - classify.features.py]: Opening connection to PostGres via Psycopg
[INFO - classify.features.py]: Loading from file: /tmp/expunge_data/expunge_features-0.csv
[INFO - classify.features.py]: Load to DB complete


CPU times: user 0 ns, sys: 8.38 ms, total: 8.38 ms
Wall time: 14.4 ms
